# Craigslist Used Cars

## Import Libraries and Dataset

In [1]:
import numpy as np
import pandas as pd

In [2]:
zf = zipfile.ZipFile('Craigslist-Used-Cars.zip')
data = pd.read_csv(zf.open('vehicles.csv'))

In [3]:
data.head()

,Unnamed: 0,id,url,region,region_url,price,year,manufacturer,model,condition,...,drive,size,type,paint_color,image_url,description,state,lat,long,posting_date
0,0,7240372487,https://auburn.craigslist.org/ctd/d/auburn-uni...,auburn,https://auburn.craigslist.org,35990,2010.0,chevrolet,corvette grand sport,good,...,rwd,NaN,other,NaN,https://images.craigslist.org/00N0N_ipkbHVZYf4...,Carvana is the safer way to buy a car During t...,al,32.590000,-85.480000,2020-12-02T08:11:30-0600
1,1,7240309422,https://auburn.craigslist.org/cto/d/auburn-201...,auburn,https://auburn.craigslist.org,7500,2014.0,hyundai,sonata,excellent,...,fwd,NaN,sedan,NaN,https://images.craigslist.org/00s0s_gBHYmJ5o7y...,I'll move to another city and try to sell my c...,al,32.547500,-85.468200,2020-12-02T02:11:50-0600
2,2,7240224296,https://auburn.craigslist.org/cto/d/auburn-200...,auburn,https://auburn.craigslist.org,4900,2006.0,bmw,x3 3.0i,good,...,NaN,NaN,SUV,blue,https://images.craigslist.org/00B0B_5zgEGWPOrt...,Clean 2006 BMW X3 3.0I. Beautiful and rare Bl...,al,32.616807,-85.464149,2020-12-01T19:50:41-0600
3,3,7240103965,https://auburn.craigslist.org/cto/d/lanett-tru...,auburn,https://auburn.craigslist.org,2000,1974.0,chevrolet,c-10,good,...,rwd,full-size,pickup,blue,https://images.craigslist.org/00M0M_6o7KcDpArw...,1974 chev. truck (LONG BED) NEW starter front ...,al,32.861600,-85.216100,2020-12-01T15:54:45-0600
4,4,7239983776,https://auburn.craigslist.org/cto/d/auburn-200...,auburn,https://auburn.craigslist.org,19500,2005.0,ford,f350 lariat,excellent,...,4wd,full-size,pickup,blue,https://images.craigslist.org/00p0p_b95l1EgUfl...,2005 Ford F350 Lariat (Bullet Proofed). This t...,al,32.547500,-85.468200,2020-12-01T12:53:56-0600


## EDA and Cleaning

#### Detecting and removing outliers 

In [ ]:
np.quantile(data.price, 0.999)

119800.0

In [ ]:
np.quantile(data.odometer, 0.99)

nan

In [ ]:
data = data[['id', 'region', 'price', 'year',
       'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'odometer',
       'title_status', 'transmission', 'drive', 'size', 'type',
       'paint_color', 'description', 'state', 'lat', 'long']]
data = data[data.price > 999]
data = data[data.price <=85000]
data = data[data.odometer <= 290098.7499999992]

In [ ]:
data = data.dropna(axis=0)

In [ ]:
data.shape

(81021, 20)

#### Re-define price column

In [ ]:
## Create a column used to collect price range information
data['price_range'] = 0

## Use 5000 as a baseline level to distribute each price range
data['price_range'] = rsm.ifelse(data.price >= 5000, 'above_5000', 'below_5000')

data['price_range'] = rsm.ifelse(
    (data.price > 5000) & (data.price <= 10000), 
    'above_5000 & below_10000', 
    data['price_range']
)

data['price_range'] = rsm.ifelse(
    (data.price > 10000) & (data.price <= 15000), 
    'above_10000 & below_15000', 
    data['price_range']
)

data['price_range'] = rsm.ifelse(
    (data.price > 15000) & (data.price <= 20000), 
    'above_15000 & below_20000', 
    data['price_range']
)

data['price_range'] = rsm.ifelse(
    (data.price > 20000) & (data.price <= 25000), 
    'above_20000 & below_25000', 
    data['price_range']
)

data['price_range'] = rsm.ifelse(
    (data.price >25000) & (data.price <= 30000), 
    'above_25000 & below_30000', 
    data['price_range']
)

data['price_range'] = rsm.ifelse(
    (data.price >30000) & (data.price <= 35000), 
    'above_30000 & below_35000', 
    data['price_range']
)

data['price_range'] = rsm.ifelse(
    (data.price >35000) & (data.price <= 40000), 
    'above_35000 & below_40000', 
    data['price_range']
)

data['price_range'] = rsm.ifelse(
    (data.price >40000) & (data.price <= 45000), 
    'above_40000 & below_45000', 
    data['price_range']
)

data['price_range'] = rsm.ifelse(
    (data.price >45000) & (data.price <= 50000), 
    'above_45000 & below_50000', 
    data['price_range']
)

data['price_range'] = rsm.ifelse(
    (data.price >50000) & (data.price <= 55000), 
    'above_50000 & below_55000', 
    data['price_range']
)

data['price_range'] = rsm.ifelse(
    (data.price >55000) & (data.price <= 60000), 
    'above_55000 & below_60000', 
    data['price_range']
)

data['price_range'] = rsm.ifelse(
    (data.price >60000) & (data.price <= 65000), 
    'above_60000 & below_65000', 
    data['price_range']
)

data['price_range'] = rsm.ifelse(
    (data.price >65000) & (data.price <= 70000), 
    'above_65000 & below_70000', 
    data['price_range']
)

data['price_range'] = rsm.ifelse(
    (data.price >70000), 
    'above_70000 ', 
    data['price_range']
)